In [ ]:
from save_csv import results_to_csv
import scipy.io as sio
import numpy as np
import numpy.random as sample
import pandas as pd
import matplotlib.pyplot as plt  
import numpy.linalg as la
from scipy.special import expit
norm = la.norm

## Collectinig Wine Data

In [ ]:
data_contents = sio.loadmat('data.mat')
X = data_contents['X']
ones = [[1] for _ in range(X.shape[0])]
X = np.append(X,ones,1)
y = np.array([val[0] for val in data_contents['y']])
X_test = data_contents['X_test']
y = np.array([[v] for v in y])
X = np.append(X,y,1)
np.random.shuffle(X)
X,y = np.array([row[:-1] for row in X]),np.array([row[-1] for row in X])  
X_train = X[:5000]
y_train = y[:5000]
X_val = X[5000:]
y_val = y[5000:]

## (a) Gradient Descent Method

In [ ]:
n = X_train.shape[0]
d = X_train.shape[1]
w = np.array([0]*d)
e = 0.0000009
lam = 0.05
alpha = 0.000001
s = np.array([expit(np.dot(X_train[i],w)) for i in range(n)])

In [ ]:
def update(X,y,w,e,s):
    w = w - e * (2*lam*w - np.matmul(X.T,y-s))
    s = np.array([expit(np.dot(X[i],w)) for i in range(n)])
    return w,s

In [ ]:
def stochastic_update(X,w,e,s,y,i):
    w = w - e * (2*lam*w - (y[i]-s[i])*X[i])
    s = np.array([expit(np.dot(X[i],w)) for i in range(n)])
    return w,s

In [ ]:
def cost(y,w,lam,s):
    Sum = sum([y[i]*np.log(s[i]) + (1-y[i])*np.log(1-s[i]) for i in range(n)])
    res = lam*np.dot(w,w) - Sum
    print('Cost: ' + str(res))
    return res

In [ ]:
def compute_accuracy(X,w,y):
    correct = 0
    total = 0
    for i in range(len(X)):
        total += 1
        a = np.dot(X[i],w)
        s = expit(a)
        p = 1 if s >= 0.5 else 0
        if p == y[i]:
            correct += 1
    return correct / total

In [ ]:
costs  = []
# working parameters: alpha = 0.0001, e = 0.0000005, lam = 0.005
prev_acc = float('inf')
acc = 1
strikes = 0
# initialize
max_acc = -float('inf')
max_w = []
w_prev = np.array([-2]*d)
while strikes < 5000:
    prev_acc = acc
    costs.append(cost(y_train,w,lam,s))
    w_prev = w
    w,s = update(X_train,y_train,w_prev,e,s)
    acc = compute_accuracy(X_val,w, y_val)
    if acc < prev_acc:
        strikes += 1
    if acc > max_acc:
        max_acc = acc
        max_w = w
    print("Accuracy: " + str(acc))


In [ ]:
print("Min Cost: " + str(min(costs)))
print("Max Accuracy: " + str(max_acc))
plt.figure()
plt.title('costs vs iterations')
plt.plot([i for i in range(len(costs))][10:], costs[10:],'r-',label='Gradient Descent Cost')
plt.xlabel('iterations')
plt.ylabel('cost')
plt.legend()

## (b) Stochastic Gradient Descent

In [ ]:
n = X_train.shape[0]
d = X_train.shape[1]
w = np.array([0]*d)
e = 0.00007
lam = 0.05
alpha = 0.00000001
s = np.array([expit(np.dot(X_train[i],w)) for i in range(n)])

In [ ]:
costs  = [[],[],[],[]]

prev_acc = float('inf')
acc = 1
strikes = 0
max_acc = [-float('inf') for _ in range(4)]
max_w = [[] for _ in range(4)]
# initialize
w_prev = np.array([-2]*d)
i = 0
for lam in [0.5,0.05,0.005,0.0005]:
    print("-"*10 + "Lambda: " + str(lam)+"-"*10)
    prev_acc = float('inf')
    j = 0
    while strikes < 200:
        costs[i].append(cost(y_train,w,lam,s))
        w_prev = w
        w,s = stochastic_update(X_train,w_prev,e,s,y_train,j)
        acc = compute_accuracy(X_val,w, y_val)
        if acc < prev_acc:
            strikes += 1
        if acc > max_acc[i]:
            max_acc[i] = acc
            max_w[i] = w
        print("Accuracy: " + str(acc))
        j += 1
        if j >= 5000:
            j = 0
        prev_acc = acc
    i += 1
    strikes = 0
    w = np.array([0]*d)

In [ ]:
for i in range(len(costs)):
    print("Min Cost: " + str((costs[i].index(min(costs[i])), min(costs[i]))))
    print("lambda: " + str(lam))
    print("Max Accuracy: " + str(max_acc[i]))
    plt.figure()
    plt.title('costs vs iterations')
    plt.plot([i for i in range(len(costs[i]))], costs[i],'r-',label='Gradient Descent Cost')
    plt.xlabel('iterations')
    plt.ylabel('cost')
    plt.legend()

## Decreasing Learning Rate

In [ ]:
n = X_train.shape[0]
d = X_train.shape[1]
w = np.array([0]*d)
e = 0.05
lam = 0.5
alpha = 0.00001
s = np.array([expit(np.dot(X_train[i],w)) for i in range(n)])

In [ ]:
costs  = []

# initialize
i = 1
e0 = e
w_prev = np.array([-2]*d)
j = 0
max_acc = -float('inf')
max_w = []
prev_acc = float('inf')
strikes = 0
diff = float('inf')
while diff > alpha:
    costs.append(cost(y_train,w,lam,s))
    w_prev = w
    w,s = stochastic_update(X_train,w_prev,e0,s,y_train,j)
    e0 = e / i
    i += 1
    j += 1
    if j >= 5000:
        j = 0
    acc = compute_accuracy(X_val,w, y_val)
    if acc < prev_acc:
            strikes += 1
    if acc < prev_acc:
        strikes += 1
    if acc > max_acc:
        max_acc = acc
        max_w = w
    prev_acc = acc
    diff = abs(norm(w)-norm(w_prev))
    print("Accuracy: " + str(acc))
    print("Diff: " + str(diff))

In [ ]:
print("Min Cost: " + str((costs.index(min(costs)), min(costs))))
print("Max Accuracy: " + str(max_acc))
# costs = [c for c in costs if c < 4000]
plt.figure()
plt.title('costs vs iterations')
plt.plot([i for i in range(len(costs))], costs,'r-',label='Gradient Descent Cost')
plt.xlabel('iterations')
plt.ylabel('cost')
plt.legend()

## Predict Method

In [1]:
def predict(X,w):
    res = []
    for i in range(len(X)):
        a = np.dot(X[i],w)
        s = expit(a)
        p = 1 if s >= 0.5 else 0
        res.append(p)
    return res